In [1]:
# Import necessary libraries
from collections import defaultdict

# Define the documents
documents = [
    "I love cats . cats are cute pets.",
    "Dogs are loyal. Dogs are good friends.",
    "Birds can sing. Birds fly in the sky.",
    "Fish live underwater. Fish come in many colors."
]

In this section, we define and create unigram models for the documents. Unigrams are single words or terms, and a unigram model represents the probability distribution of individual terms in the document. The unigram_model function counts the occurrences of each term in a document, calculates the probabilities, and returns the unigram model. We create unigram models for all documents in the collection.

# Create Unigram Models

In [2]:
def unigram_model(document):
    words = document.split()
    total_words = len(words)
    unigram_counts = defaultdict(int)   
    for word in words:
        unigram_counts[word] += 1
    unigram_model = {word: count / total_words for word, count in unigram_counts.items()}
    return unigram_model

In [3]:
# Create unigram models for all documents
unigram_models = [unigram_model(doc) for doc in documents]

In [4]:
unigram_models

[{'I': 0.125,
  'love': 0.125,
  'cats': 0.25,
  '.': 0.125,
  'are': 0.125,
  'cute': 0.125,
  'pets.': 0.125},
 {'Dogs': 0.2857142857142857,
  'are': 0.2857142857142857,
  'loyal.': 0.14285714285714285,
  'good': 0.14285714285714285,
  'friends.': 0.14285714285714285},
 {'Birds': 0.25,
  'can': 0.125,
  'sing.': 0.125,
  'fly': 0.125,
  'in': 0.125,
  'the': 0.125,
  'sky.': 0.125},
 {'Fish': 0.25,
  'live': 0.125,
  'underwater.': 0.125,
  'come': 0.125,
  'in': 0.125,
  'many': 0.125,
  'colors.': 0.125}]

In [5]:
#we have a query 
query = "I like cats and dogs"

In [6]:
def calculate_query_probability(query, document_model):
    # Tokenize the query into words
    query_words = query.split()
    
    # Initialize the probability for the entire query
    query_probability = 1.0
    
    # Calculate the probability for each term in the query
    for word in query_words:
        if word in document_model:
            query_probability *= document_model[word]
        else:
            query_probability = 0.0
            break
    
    return query_probability

In [7]:
query_probability = calculate_query_probability(query, unigram_models[0])

In [8]:
query_probability

0.0

## Your task

In [62]:
# Use Laplace Smoothing for this problem

# https://www.exploredatabase.com/2020/10/explain-add-1-laplace-smoothing-with-example.html

def laplace_smoothing_unigram(doc,query):
    total_words = 0
    word_counts = {}
    vocabulary = set()

    # Count the occurrences of each word in the corpus
    words = doc.lower().split()
    total_words = len(words)
    for word in words:
        word_counts[word] = word_counts.get(word, 0) + 1
        vocabulary.add(word)

    # Calculate the Laplace smoothed probability for each word in the query
    query_words = query.lower().split()
    prob = 1.0
    for word in query_words:
        count = word_counts.get(word, 0)
        prob *= (count + 1) / (total_words + len(vocabulary))
    return prob

smoothed_unigram_model = {f"Doc {index}":format(laplace_smoothing_unigram(doc,query),'.8f') for index,doc in enumerate(documents)}

print(smoothed_unigram_model)

{'Doc 0': '0.00000790', 'Doc 1': '0.00001206', 'Doc 2': '0.00000132', 'Doc 3': '0.00000132'}


In [64]:
# create a bigram model and apply smoothing model
# Define a function to calculate Laplace-smoothed bigram models
def laplace_smoothed_bigram_model(document):
    words = document.split()
    total_words = len(words)
    bigram_counts = defaultdict(int)
    
    # Initialize all bigrams with a count of 1 (Laplace smoothing)
    for i in range(total_words - 1):
        bigram = (words[i], words[i + 1])
        bigram_counts[bigram] += 1
    
    # Calculate Laplace-smoothed probabilities
    bigram_model = {bigram: (count + 1) / (total_words + total_words) for bigram, count in bigram_counts.items()}
    return bigram_model

# Create Laplace-smoothed bigram models for all documents
laplace_smoothed_bigram_models = [laplace_smoothed_bigram_model(doc) for doc in documents]

# Print Laplace-smoothed bigram models
for i, model in enumerate(laplace_smoothed_bigram_models):
    print(f"Document {i + 1} Laplace-Smoothed Bigram Model:")
    for bigram, probability in model.items():
        print(f"{bigram[0]} {bigram[1]}: {probability:.4f}")
    print()

def calculate_query_probability(query, document_model):
    query_words = query.split()
    query_bigrams = [(query_words[i], query_words[i + 1]) for i in range(len(query_words) - 1)]
    
    query_probability = 1.0
    
    for bigram in query_bigrams:
        # Apply Laplace smoothing for unseen bigrams using the document_model
        bigram_probability = document_model.get(bigram, 1 / (len(document_model) + len(laplace_smoothed_bigram_models[0])))
        query_probability *= bigram_probability

    return query_probability

# Calculate query probability for each document
query_probabilities = []

for document_model in laplace_smoothed_bigram_models:
    probability = calculate_query_probability(query, document_model)
    query_probabilities.append(probability)

# Find the document with the highest probability for the query
most_possible_document_index = query_probabilities.index(max(query_probabilities))
most_possible_document = documents[most_possible_document_index]

print("Query Bigram Probabilities for Each Document:")
for i, probability in enumerate(query_probabilities):
    print(f"Document {i + 1}: {probability:.6f}")

print("The most possible document for the query is:", most_possible_document)

Document 1 Laplace-Smoothed Bigram Model:
I love: 0.1250
love cats: 0.1250
cats .: 0.1250
. cats: 0.1250
cats are: 0.1250
are cute: 0.1250
cute pets.: 0.1250

Document 2 Laplace-Smoothed Bigram Model:
Dogs are: 0.2143
are loyal.: 0.1429
loyal. Dogs: 0.1429
are good: 0.1429
good friends.: 0.1429

Document 3 Laplace-Smoothed Bigram Model:
Birds can: 0.1250
can sing.: 0.1250
sing. Birds: 0.1250
Birds fly: 0.1250
fly in: 0.1250
in the: 0.1250
the sky.: 0.1250

Document 4 Laplace-Smoothed Bigram Model:
Fish live: 0.1250
live underwater.: 0.1250
underwater. Fish: 0.1250
Fish come: 0.1250
come in: 0.1250
in many: 0.1250
many colors.: 0.1250

Query Bigram Probabilities for Each Document:
Document 1: 0.000026
Document 2: 0.000048
Document 3: 0.000026
Document 4: 0.000026
The most possible document for the query is: Dogs are loyal. Dogs are good friends.
